In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import html5lib
import psycopg2
import pickle

ImportError: No module named bs4

In [20]:
print 'Done'

Done


##Format Desired for Database

## Functions to get database information

###Generate test soup objects to run with functions

In [3]:
test_climb = '/v/northeast-arete/105732404'

base_url = 'http://mountainproject.com'

#Climb obeserved features test soup
test_soup = BeautifulSoup(requests.get(base_url+test_climb).content)

#Climb user enetered features test soup
climb_id_str = test_climb.split('/')[-1]
user_url = base_url+'/scripts/ShowObjectStats.php?id='+climb_id_str
user_content = requests.get(user_url).content
test_soup_user = BeautifulSoup(user_content)

In [4]:
def get_climb_data(soup):
    #Get obsereved features from the main page of a climb

    climb_name = soup.select('[itemprop="itemreviewed"]')[0].text
    yds = soup.findAll('div',{'id':'rspCol800'})[0].select('.rateYDS')[0].text.split()[-1]
    yds_consensus = soup.findAll('div',{'id':'rspCol800'})[0].select('.rateYDS')[1].text.split()[-1]

    stars = float(soup.select('#starSummaryText [itemprop="rating"] meta')[0]['content'])
    star_votes = soup.findAll('span',{'id': 'starSummaryText'} )[0] \
                    .findAll('meta',{'itemprop':'votes'})[0]['content']
    climb_type = soup.select('.rspCol table')[2].select('td')[1].text
    location = ','.join([a.text for a in soup.select('#navBox [itemtype="http://data-vocabulary.org/Breadcrumb"]')])
    return climb_name, location, climb_type, yds, yds_consensus, stars, star_votes

(climb_name, location, climb_type, yds,
 yds_consensus, stars, star_votes) = get_climb_data(test_soup)
print climb_name
print climb_name
print location
print climb_type
print 'yds:',yds
print 'yds_consensus:', yds_consensus
print "stars:", stars
print "stars_votes:",star_votes

Northeast Arete
Northeast Arete
Nevada,Red Rock,Icebox Canyon,Bridge Mountain
Trad, 5 pitches, 800'
yds: 5.6
yds_consensus: 5.5
stars: 2.8
stars_votes: 16


In [5]:
def get_comments(soup):
    comment_boxes = soup.select('.comvis')
    text = []
    user_ids = []
    for box in comment_boxes:
        text += [str(box).split('<!--MPTEXT-->')[1].split('<!--MPTEXTEND-->')[0]]
        user_ids += [box.find('a')['href'].split('//')[-1]]
    return text, user_ids
text, user_ids = get_comments(test_soup)
print user_ids
text

['10644', '10146', '11453', '10445', '106310836', '105919888', '10445', '106893421', '105951192', '105919888', '106314757', '10445', '106314757', '106095990']


["George:  You truly are an adventuresome soul. Not many people would endeavor to put in the effort to explore the further reaches of Red Rock Canyons. Subsequently, not many others would discover the secret treasures like arches and hidden forests. There is so much to discover out there once you get away from the scenic loop road. My hat's off to you. I have not yet, but have long wanted to do a route on Bridge Mountain as it is such a strikingly beautiful swath of stone. Thanks for posting so many good routes and concise information. Philop.s. Your digi-pic of Wilson is awesome you should think about posting it.",
 'Thanks for the comments, Philo!  Regarding this route, you probably do not want to take a skeletal rack if you use approaches 1 or 2, as they both involve climbing of at least 5.8.  The shortest route up the Refrigerator Buttress is "Burlesque (5.9)",  for which you need some big cams, I believe.  Bummer to have to haul all those heavy cams over the summit.',
 'This is a 

In [6]:
def get_stars(soup):
    #get individual star ratings by user
    table = None
    for heading in soup.select('span.dkorange.bold'):
        if heading.text == 'Star Quality Votes':
            table = heading.find_next_sibling('table')
    
    user_names = []
    user_id = []
    stars = []
    
    for i,user_link in enumerate(table.select('a[href^="/u/"]')):
        #print i
        user_names += [user_link.text]
        script = user_link.parent.next_sibling.select('script')[0]
        #print [script.text.split('starsHtml(')[1][0] ]
        stars += [script.text.split('starsHtml(')[1][0] ]
        user_id += [user_link['href'].split('//')[-1]]
    return user_names, user_id, stars
user_names, user_ids, stars = get_stars(test_soup_user)
print user_names
print user_ids
print stars

[u'Royal', u'Peter Lewis', u'Howard Snell', u'George Bell', u'Alan Butler', u'Jim R', u'Steven Tata', u'Zeb Rafaker', u'Orphaned', u'J. Thompson', u'cassondra', u'sqwirll', u'Crotch Robbins', u'Brent Butler', u'John Hegyes', u'Crotch Robbins']
['106794218', '106586290', '106775085', '10146', '106688497', '105951192', '107548227', '106119950', '105990845', '10445', '106310836', '105805486', '11331', '106738886', '11591', '11331']
[u'5', u'5', u'5', u'4', u'4', u'4', u'4', u'4', u'4', u'4', u'3', u'3', u'3', u'3', u'3', u'3']


In [7]:
def get_ratings(soup):
    #Returns the individuals ratings by user for a climb.
    for heading in soup.select('span.dkorange.bold'):
        #print heading
        if heading.text == 'Suggested Ratings':
            table = heading.find_next_sibling('table')
    user_names = []
    user_ids = []
    ratings = []
    additionals = []
    
    for user_link in table.select('a[href^="/u/"]'):
        user_names += [user_link.text]
        ratings += [user_link.parent.next_sibling.select('.rateYDS')[0].text]
        additionals += [user_link.parent.nextSibling.contents[-1]]
        user_ids += [user_link['href'].split('//')[-1]]
    return user_names, user_ids, ratings, additionals
user_names, user_ids, ratings, additionals = get_ratings(test_soup_user)
print user_names
print user_ids
print ratings
print additionals

[u'John Hegyes', u'Zeb Rafaker', u'Peter Lewis', u'Royal', u'sqwirll', u'Orphaned', u'George Bell']
['11591', '106119950', '106586290', '106794218', '105805486', '105990845', '10146']
[u'5.6', u'5.5', u'5.5', u'5.5', u'5.4', u'Easy 5th', u'5.6']
[u' ', u' ', u' ', u' ', u' ', u' ', u' ']


In [8]:
def get_ticks(soup):
    #Returns user tick dates.  
    table = None
    for heading in soup.select('span.dkorange.bold'):
        if heading.text == 'Ticks':
            table = heading.find_next_sibling('table')
    user_names = []
    user_ids = []
    tick_dates = []
    
    for user_link in table.select('a[href^="/u/"]'):
        user_names += [user_link.text]
        user_ids += [user_link['href'].split('//')[-1]]
        tick_dates += [user_link.parent.next_sibling.text.strip()]
        
    return user_names, user_ids, tick_dates
user_names, user_ids, tick_dates= get_ticks(test_soup_user)

print user_names
print user_ids
print tick_dates

[u'Steven Tata', u'David Carrier', u'Alan Butler', u'Brent Butler', u'Royal', u'Howard.', u'Nick Hamill', u'Mark Beauchamp', u'Sue B', u'cassondra', u'Peter Lewis', u'ephrat', u'Crotch Robbins', u'cassondra', u'Howard Snell', u'Darren in Vegas', u'sqwirll', u'Josh Hensley', u'J. Thompson', u'John Hegyes', u'wayniak']
['107548227', '109340087', '106688497', '106738886', '106794218', '106886459', '106993267', '106344483', '106710164', '106310836', '106586290', '107545625', '11331', '106310836', '106775085', '105796642', '105805486', '106417489', '10445', '11591', '11453']
[u'Apr 17, 2015', u'Nov 20, 2014', u'Oct 18, 2014', u'Oct 18, 2014', u'Oct 9, 2014', u'Apr 28, 2014', u'Oct 15, 2012', u'Sep 2, 2012', u'Sep 2, 2012', u'Jul 25, 2012', u'May 14, 2012', u'Mar 30, 2012', u'Jan 15, 2012', u'Nov 17, 2011', u'Oct 9, 2011', u'Sep 11, 2011', u'Nov 8, 2010', u'May 7, 2009', u'Mar 17, 2009', u'Aug 9, 2006', u'Apr 9, 2006']


In [9]:
def get_climbs_mpsearch(url, page_num=1, show_len=False, name=False):
    if name:
        try:
            climb_list = pickle.load(open('climb_list_'+name+'.p','rb'))
            print 'climb_list_'+name+' loaded'
            if show_len:
                print "Number of climbs:\n", len(climb_list)
            return climb_list
        except:
            pass
    climb_list = []
    for i in range(page_num):
        soup = BeautifulSoup(requests.get(url+"&page="+str(i+1)).content)
        climb_table = soup.findAll('table', {'class':'objectList'})[1]
        climb_list += [row.findNext('a')['href'] for row in climb_table.findAll('tr')[1:]]
    if show_len:
        print "Number of climbs:\n", len(climb_list)
    if name:
        with open('climb_list_'+name+'.p','w') as f:
            pickle.dump(climb_list, f)
    return climb_list


In [10]:
big_cotten_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105739280%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
big_cotten_climb_list = get_climbs_mpsearch(big_cotten_url, 8, True, 'big_cotten')

climb_list_big_cotten loaded
Number of climbs:
362


In [10]:
climb_list = []

american_fork_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105739274%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
american_fork_n = 5
american_fork_climb_list = get_climbs_mpsearch(american_fork_url, 5, True, 'american_fork')

zion_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105716799%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
zion_n = 2
zion_climb_list = get_climbs_mpsearch(zion_url, 2, True, 'zion')

rifle_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105802424%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
rifle_n = 6
rifle_climb_list = get_climbs_mpsearch(rifle_url, 6, True, 'rifle')

boulder_can_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105744222%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
boulder_can_n = 18
boulder_can_climb_list = get_climbs_mpsearch(boulder_can_url, 18, True, 'boulder')

elderado_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105744246%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
elderado_n = 9
elderado_climb_list = get_climbs_mpsearch(elderado_url, 9, True, 'elderado')

flatiron_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105797700%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
flatiron_n = 7
flatiron_climb_list = get_climbs_mpsearch(flatiron_url, 7, True, 'flatiron')

shelf_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105800427%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
shelf_n = 12
shelf_climb_list = get_climbs_mpsearch(shelf_url, 12, True, 'shelf')

climb_list_american_fork loaded
Number of climbs:
246
climb_list_zion loaded
Number of climbs:
81
Number of climbs:
265
Number of climbs:
877
Number of climbs:
440
Number of climbs:
314
Number of climbs:
588


In [11]:
canada_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105881369%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
canada_n = 10
canada_climb_list = get_climbs_mpsearch(canada_url, 10, True, 'canada')

mexico_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105910759%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
mexico_n =4
mexico_climb_list = get_climbs_mpsearch(mexico_url, 4, True, 'mexico')

europe_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27106660030%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
europe_n = 4
europe_climb_list = get_climbs_mpsearch(europe_url, 4, True, 'europe')

wy_trad_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708960%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&stars=0&pitches=0&sort1=area&sort2=rating'
wy_trad_n = 9
wy_trad_climb_list = get_climbs_mpsearch(wy_trad_url, 9, True, 'wy_trad')

wy_sport_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708960%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
wy_sport_n = 14
wy_sport_climb_list = get_climbs_mpsearch(wy_sport_url, 14, True, 'wy_sport')

sd_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708963%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
sd_n = 12
sd_climb_list = get_climbs_mpsearch(sd_url, 12, True, 'sd')

id_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708958%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
id_n = 12
id_climb_list = get_climbs_mpsearch(id_url, 12, True, 'id')

mt_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105907492%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
mt_n = 4
mt_climb_list = get_climbs_mpsearch(mt_url, 4, True, 'mt')

me_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105948977%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
me_n = 3
me_climb_list = get_climbs_mpsearch(me_url, 3, True, 'me')

climb_list = wy_sport_climb_list+sd_climb_list+id_climb_list+mt_climb_list+me_climb_list


climb_list_canada loaded
Number of climbs:
493
climb_list_mexico loaded
Number of climbs:
200
climb_list_europe loaded
Number of climbs:
153
climb_list_wy_trad loaded
Number of climbs:
410
climb_list_wy_sport loaded
Number of climbs:
651
climb_list_sd loaded
Number of climbs:
566
climb_list_id loaded
Number of climbs:
583
climb_list_mt loaded
Number of climbs:
195
climb_list_me loaded
Number of climbs:
112


In [17]:
wa_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708966%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
wa_n = 20
wa_climb_list = get_climbs_mpsearch(wa_url, 20, True, 'wa')

Number of climbs:
953


In [18]:
con = None
con = psycopg2.connect(database='mountainproject', user='User')
cur = con.cursor()
con.autocommit = True

In [19]:
#climbs = get_climbs(base_url, walls)
#Metamorphosis, /v/masterpiece/105878204
cur.execute('SELECT climb_id FROM tb_climb')
done_climbs = set([r[0] for r in cur])
error_list = []

climbs = wa_climb_list
for j,climb in enumerate(climbs): 
    try: 
    #if True:
        print j, 'Trying climb:', climb
        climb_id_str = climb.split('/')[-1]
        climb_id = int(climb_id_str)
        if climb_id in done_climbs:
            print "Have climb already."
            continue    
        soup = BeautifulSoup(requests.get(base_url+climb).content)

        #Collecting data
        #Climb data
        (climb_name, location, climb_type, yds,
         yds_consensus, stars, star_votes) = get_climb_data(soup)
        #Comment data
        user_comments_tup = get_comments(soup)

        #User data
        user_url = base_url+'/scripts/ShowObjectStats.php?id='+climb_id_str
        soup_user = BeautifulSoup(requests.get(user_url).content)
        user_stars_tup = get_stars(soup_user)
        user_ratings_tup = get_ratings(soup_user)
        user_ticks_tup = get_ticks(soup_user)

        #Adding to databases

        #Writing climb data

        cur.execute("INSERT INTO tb_climb VALUES(%s, %s, %s, %s, %s, %s, %s, %s)",
                    (climb_id,
                     climb_name,
                     location,
                     climb_type,
                     yds, 
                     yds_consensus,
                     stars,
                     star_votes
                    ))

        print 'Writing comments'
        for i in range(len(user_comments_tup[0])):
            cur.execute("INSERT INTO comments VALUES(%s, %s, %s, %s)",
                        (user_comments_tup[1][i],
                         climb_name,
                         climb_id,
                         user_comments_tup[0][i]
                        ))

        print 'Writing user star data'
        for i in range(len(user_stars_tup[0])):
            cur.execute("INSERT INTO stars VALUES(%s, %s, %s, %s, %s)",
                    (user_stars_tup[1][i],
                     climb_name,
                     user_stars_tup[0][i],
                     climb_id,
                     user_stars_tup[2][i]
                    ))    

        print 'Writing user ratings data'
        for i in range(len(user_ratings_tup[0])):
            cur.execute("INSERT INTO ratings VALUES(%s, %s, %s, %s, %s, %s)",
                    (user_ratings_tup[1][i],
                     climb_name,
                     user_ratings_tup[0][i],
                     climb_id,
                     user_ratings_tup[2][i],
                     user_ratings_tup[3][i]
                    ))

        print 'Writing user tick data'
        for i in range(len(user_ticks_tup[0])):
            cur.execute("INSERT INTO ticks VALUES(%s, %s, %s, %s, %s)",
                    (user_ticks_tup[1][i],
                     climb_name,
                     user_ticks_tup[0][i],
                     climb_id,
                     user_ticks_tup[2][i]
                    ))
        con.commit()

        done_climbs.add(climb)
        print 'Climb Added!'
    except:
        error_list += [climb]
        print "Climb NOT added."
    time.sleep(1.5*(1+np.random.rand()))
print "Done!"

0 Trying climb: /v/south-east-corner/105800483
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
1 Trying climb: /v/little-wing/107221659
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
2 Trying climb: /v/cruisin/107781447
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
3 Trying climb: /v/free-for-all/108372532
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
4 Trying climb: /v/right-gull/109481319
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
5 Trying climb: /v/jills-thrill/106985373
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb Added!
6 Trying climb: /v/young-warriors/107745977
Writing comments
Writing user star data
Writing user ratings data
Writing user tick data
Climb 

In [560]:
len(error_list)

1

##Get climbs by mp search results

In [629]:
#Get all climbs with at least 5 votes in NH

nh_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=10&selectedIds=%27105872225%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
nh_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=10&selectedIds=%27105872225%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating&page=1'
nh_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105872225%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating&page='
nh_climb_list = []
for i in range(17):
    nh_soup = BeautifulSoup(requests.get(nh_url+str(i+1)).content)
    climb_table = nh_soup.findAll('table', {'class':'objectList'})[1]
    nh_climb_list += [row.findNext('a')['href'] for row in climb_table.findAll('tr')[1:]]

In [640]:
ky_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105868674%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating&page='
ky_climb_list = []
for i in range(14):
    ky_soup = BeautifulSoup(requests.get(ky_url+str(i+1)).content)
    climb_table = ky_soup.findAll('table', {'class':'objectList'})[1]
    ky_climb_list += [row.findNext('a')['href'] for row in climb_table.findAll('tr')[1:]]

In [628]:
climb_table = nh_soup.findAll('table', {'class':'objectList'})[1]
nh_climb_list += [row.findNext('a')['href'] for row in climb_table.findAll('tr')[1:]]

In [641]:
len(ky_climb_list)

674

In [5]:
#Climb urls from search results
nv_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105708961%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
nv_climbs =get_climbs_mpsearch(nv_url, 19, True)


Number of climbs:
945


In [31]:
maple_can_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105739298%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
maple_can_climb_list = get_climbs_mpsearch(maple_can_url, 6, True)

Number of climbs:
282


In [9]:
moab_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105716711%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
moab_climb_list = get_climbs_mpsearch(moab_url, 17, True)

Number of climbs:
809


In [9]:
little_cotten_url = 'http://www.mountainproject.com/scripts/Search.php?searchType=routeFinder&minVotes=5&selectedIds=%27105739277%27&type=rock&diffMinrock=1000&diffMinboulder=20000&diffMinaid=70000&diffMinice=30000&diffMinmixed=50000&diffMaxrock=12300&diffMaxboulder=21400&diffMaxaid=75260&diffMaxice=38500&diffMaxmixed=60000&is_trad_climb=1&is_sport_climb=1&is_top_rope=1&stars=0&pitches=0&sort1=area&sort2=rating'
little_cotten_climb_list = get_climbs_mpsearch(little_cotten_url,9,'little_cotten')

Number of climbs:
430


In [12]:
base_url = 'http://mountainproject.com'

In [13]:
rumney = 'v/rumney/105867829'

In [298]:
rumney_soup = BeautifulSoup(requests.get(base_url+'/'+rumney).content)

In [299]:
#walls= [a['href'] for a in rumney_soup.select(\
#        '#mpbox1995446815 br ~ a[href^="/v/"]')]

In [6]:
climb = '/v/waimea/105867832'

area_soup = BeautifulSoup(requests.get(base_url+climb).content)

def get_areas(area_soup):
    
    #check if climb
    
    walls= [a['href'] for a in area_soup.select(\
        '#viewerLeftNavColContent > div:nth-of-type(1)   a[href^="/v/"]')]
    return walls
get_areas(area_soup)

NameError: name 'base_url' is not defined

In [7]:
def get_climbs(base_url, walls): 
    wall_soup = {}

    #climbs = [a['href'] for a in wall_soup[wall].select(\
    #        '#viewerLeftNavColContent > div:nth-of-type(1)   a[href^="/v/"]')]
    climbs = []
    for wall in walls:
        time.sleep(2)
        print 'wall:', wall
        wall_soup[wall] = BeautifulSoup(requests.get(base_url+wall).content)
        climbs += [a['href'] for a in wall_soup[wall].select(\
            '#viewerLeftNavColContent > div:nth-of-type(1)   a[href^="/v/"]')][1:]

        #climbs = wall_soup[wall].select('#viewerLeftNavColContent > div:nth-of-type(1)   a[href^="/v/"]')
    return climbs
climbs = get_climbs(base_url, walls[4:9])
climbs

NameError: name 'base_url' is not defined

In [583]:
a = climb1_soup.findAll('table',{'cellpadding':'0'})

In [595]:
#print user_content

In [ ]:
#print(soup_user.prettify())

In [ ]:
#print(soup_user.prettify())

##Climb Data

In [126]:
soup = BeautifulSoup(requests.get('http://www.mountainproject.com/v/critical-view/105715007').content)

In [17]:
print climb_name
print climb_id
print climb_name
print location
print climb_type
print 'yds:',yds
print 'yds_consensus:', yds_consensus
print "stars:", stars
print "stars_votes:",star_votes


Waimea
105867858
Waimea
New Hampshire,Rumney,Waimea
Sport, 1 pitch, 50'
yds: 5.10d
yds_consensus: 5.10d
stars: 3.6
stars_votes: 181


##PostgreSQL

In [11]:
import psycopg2

In [545]:
con = None
con = psycopg2.connect(database='mountainproject', user='User')
cur = con.cursor()
con.autocommit = True

In [510]:
def reset_tb_climb(cur, con):
    print "You don't want to do this."
    return
    try:
        cur.execute("DROP TABLE tb_climb")
    except:
        pass
    try:
        cur.execute('CREATE TABLE tb_climb(climb_id INTEGER PRIMARY KEY, \
                                       climb_name TEXT, \
                                       location TEXT, \
                                       type_text TEXT,\
                                       yds VARCHAR(10), \
                                       yds_consensus VARCHAR(10), \
                                       stars FLOAT, \
                                       star_votes INT)')
    except:
        pass

    
    cur.execute("INSERT INTO tb_climb VALUES(%s, %s, %s, %s, %s, %s, %s, %s)",
                (111,
                 'Some climb',
                 'some place',
                 'sport',
                 '5.11a', 
                 '5.6',
                 4.2,
                 36
                ))
    con.commit()
reset_tb_climb(cur, con)

In [511]:
def reset_stars(cur, con):
    print "You don't want to do this."
    return
    try:
        cur.execute("DROP TABLE stars")
    except:
        print 'didnt drop'
    try:
        cur.execute('CREATE TABLE stars(user_id INTEGER, \
                                       climb_name TEXT, \
                                       user_name TEXT, \
                                       climb_id INT,\
                                       user_stars INT)')
        
    except:
        print 'didnt create'
        
    cur.execute("INSERT INTO stars VALUES(%s, %s, %s, %s, %s)",
                (111,
                 'some climb',
                 'some dude',
                 222,
                 4
                ))
    con.commit()
reset_stars(cur, con)


In [512]:
def reset_ratings(cur, con):
    print "You don't want to do this."
    return
    try:
        cur.execute("DROP TABLE ratings")
    except:
        print 'didnt drop'
    try:
        cur.execute('CREATE TABLE ratings(user_id INTEGER, \
                                       climb_name TEXT, \
                                       user_name TEXT, \
                                       climb_id INT,\
                                       user_rating VARCHAR(10),\
                                       user_risk VARCHAR(10))')
        
    except:
        print 'didnt create'
        
    cur.execute("INSERT INTO ratings VALUES(%s, %s, %s, %s, %s, %s)",
                (111,
                 'some climb',
                 'some dude',
                 222,
                 '5.11a',
                 'PG13'
                ))
    con.commit()
reset_ratings(cur, con)

In [513]:
def reset_ticks(cur, con):
    print "You don't want to do this."
    return
    try:
        cur.execute("DROP TABLE ticks")
    except:
        print 'didnt drop'
    try:
        cur.execute('CREATE TABLE ticks(user_id INTEGER, \
                                       climb_name TEXT, \
                                       user_name TEXT, \
                                       climb_id INT,\
                                       tick_date TEXT)')
        
    except:
        print 'didnt create'
        
    cur.execute("INSERT INTO ticks VALUES(%s, %s, %s, %s, %s)",
                (111,
                 'some climb',
                 'some dude',
                 222,
                 '1998-03-23'
                ))
    con.commit()
reset_ticks(cur, con)

In [514]:
def reset_comments(cur, con):
    print "You don't want to do this."
    try:
        cur.execute("DROP TABLE comments")
    except:
        print 'didnt drop'
    try:
        cur.execute('CREATE TABLE comments(user_id INTEGER, \
                                       climb_name TEXT, \
                                       climb_id INT,\
                                       comment TEXT)')
        
    except:
        print 'didnt create'
        
    cur.execute("INSERT INTO comments VALUES(%s, %s, %s, %s)",
                (111,
                 'some climb',
                 222,
                 'This is the most scary climb EVAH!!'
                ))
    con.commit()
reset_comments(cur, con)

In [515]:
df_climb = pd.DataFrame({
        'climb': ['A Brief History of Clim'],
        'wall': ['The Gallery'],
        'sub_area': ['Pendergrass-Murray Recreational Preserve (PMRP)'],
        'area': ['Red River Gorge'],
        'state': ['Kentucky'],
        'climb_id': [106125070],
        'yds': ['5.10c'],
        'yds_consensus': ['5.10b'],
        'stars': [3.6],
        'star_vots': [211]
    })

df_stars = pd.DataFrame({
        'climb': ['A Brief History of Climb'],
        'climb_id': [106125070],
        'user': ['Xzavier Thompson'],
        'user_id': [107495403],
        'stars': [4]     
    })

df_rating = pd.DataFrame({
        'climb': ['A Brief History of Climb'],
        'climb_id': [106125070],  
        'user': ['Brandon Marks'],
        'user_id': [1070559853],
        'rating': ['5.10c']
    })

df_ticks = pd.DataFrame({
        'climb': ['A Brief History of Climb'],
        'climb_id': [106125070],
        'user': ['KevinLarson'],
        'user_id': ['110765384'],
        'tick':['Jul 18, 2015']
    })